# Etude - Piano Cover Generation

This notebook allows you to run the Etude pipeline directly from Google Drive.

## Setup

1.  **Mount Google Drive**: This allows the notebook to access your files.
2.  **Install Dependencies**: Installs the necessary libraries (Demucs, PyTorch, Madmom, etc.).
3.  **Download Models**: Downloads the pre-trained checkpoints required for inference.
4.  **Navigate to Project**: Changes the working directory to where you cloned/uploaded the Etude repository.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# --- CONFIGURATION ---
# Change this path to match where you uploaded/cloned the Etude folder in your Drive.
# Example: '/content/drive/MyDrive/Projects/Etude'
PROJECT_PATH = '/content/drive/MyDrive/Etude'
# ---------------------

if not os.path.exists(PROJECT_PATH):
    print(f"WARNING: Path {PROJECT_PATH} does not exist. Please check your Drive structure.")
else:
    os.chdir(PROJECT_PATH)
    print(f"Current working directory: {os.getcwd()}")

## Install Dependencies
We need to install system libraries (ffmpeg) and Python packages.

**Note on Dependencies:**
The `synctoolbox` library requires `numpy<2.0` and `pandas<2.0`. We explicitly install these older versions to ensure compatibility. This might cause pip to show warnings about `google-colab` or other pre-installed packages being incompatible, but this is expected and necessary for the audio processing pipeline to work.

In [ ]:
!sudo apt-get update && sudo apt-get install -y ffmpeg

In [ ]:
# Ensure Cython is available for building madmom, but don't force a version.
!pip install Cython

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Install Madmom from source with patching
# We clone the repo, patch the source code (fixing np.int/np.float issues), and then install.
# This ensures the Cython extensions are compiled with the correct NumPy types.

import os
if not os.path.exists('madmom'):
    # IMPORTANT: Use --recursive to fetch the 'models' submodule!
    !git clone --recursive https://github.com/CPJKU/madmom.git

# Run the patch script on the SOURCE directory
!python fix_madmom.py --path madmom

# Install from the patched source
!pip install ./madmom

In [ ]:
# Verify Installation & Versions
import torch
import torchvision
import torchaudio
import numpy as np
import madmom

print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Torchaudio version: {torchaudio.__version__}")
print(f"Madmom version: {madmom.__version__}")

print("\n--- Verifying Madmom Fixes ---")
try:
    # Test 1: Imports (Fixes MutableSequence)
    from madmom.features.beats import DBNBeatTrackingProcessor
    from madmom.features.downbeats import DBNDownBeatTrackingProcessor
    print("✅ Imports successful!")

    # Test 2: Initialization (Fixes np.int in compiled code)
    beat_processor = DBNBeatTrackingProcessor(fps=100)
    downbeat_processor = DBNDownBeatTrackingProcessor(beats_per_bar=4, fps=100)
    print("✅ Processors initialized successfully!")

    # Test 3: Dummy Inference
    dummy_beat_activation = np.random.rand(1000)
    beats = beat_processor(dummy_beat_activation)
    print(f"✅ Inference successful! Detected {len(beats)} dummy beats.")

except Exception as e:
    print(f"❌ Verification failed: {e}")
    raise e

## Download Pre-trained Models
This step downloads the necessary model checkpoints and extracts them to the `checkpoints/` directory.

In [ ]:
import os

if not os.path.exists('checkpoints'):
    print("Downloading checkpoints...")
    !wget -O checkpoints.zip "https://github.com/Xiugapurin/Etude/releases/download/latest/checkpoints.zip"
    !unzip -q checkpoints.zip
    !rm checkpoints.zip
    print("Checkpoints downloaded and extracted.")
else:
    print("Checkpoints directory already exists. Skipping download.")

## Run Pipeline

You can now run the data preparation or inference scripts.

### Inference (Generate Piano Cover)
Use `infer.py` to generate a cover from a YouTube URL or local file.

In [ ]:
# Example: Generate a cover from a YouTube URL
# Replace the URL with your desired song.
!python infer.py --input "https://www.youtube.com/watch?v=dQw4w9WgXcQ" --output_name "my_cover"

### Resuming from a Crash
If the pipeline crashes (e.g., during Stage 2), you can resume from a specific stage using `--start-from`.

- `extract`: Start from beginning (Stage 1)
- `structuralize`: Start from Stage 2 (skips extraction)
- `decode`: Start from Stage 3 (skips extraction and beat detection)

In [ ]:
# Example: Resume from Stage 2 (Structuralize)
# !python infer.py --input "https://www.youtube.com/watch?v=dQw4w9WgXcQ" --output_name "my_cover" --start-from structuralize

## Visualization and Playback
Run these cells to listen to your generated MIDI and view it as sheet music.

In [ ]:
# Install dependencies for playback (FluidSynth) and visualization (LilyPond)
!sudo apt-get install -y fluidsynth fluid-soundfont-gm lilypond
!pip install pyfluidsynth music21

In [ ]:
import pretty_midi
from IPython.display import Audio, display
import numpy as np

def play_midi(midi_path):
    print(f"Synthesizing {midi_path}...")
    pm = pretty_midi.PrettyMIDI(str(midi_path))
    # Synthesize audio using the installed SoundFont
    # Sampling rate 44100Hz
    audio_data = pm.fluidsynth(fs=44100)
    display(Audio(audio_data, rate=44100))

# Replace with your actual output filename
output_midi = "outputs/inference/my_cover.mid"
if os.path.exists(output_midi):
    play_midi(output_midi)
else:
    print(f"File not found: {output_midi}")

In [ ]:
from music21 import converter, environment
from IPython.display import Image, display
import glob

# Configure music21 to use LilyPond
us = environment.UserSettings()
us['lilypondPath'] = '/usr/bin/lilypond'

def show_sheet_music(midi_path, start_measure=None, end_measure=None):
    print(f"Rendering sheet music for {midi_path}...")
    try:
        s = converter.parse(str(midi_path))
        
        # Filter measures if requested
        if start_measure is not None and end_measure is not None:
            s_to_render = s.measures(start_measure, end_measure)
            print(f"Rendering measures {start_measure} to {end_measure}...")
        else:
            s_to_render = s
            print("Rendering full score (this may take a moment)...")
        
        # Render to image(s)
        # music21 with lilypond backend generates 'lily.png' or 'lily-page1.png', 'lily-page2.png' etc.
        # We use a base filename and check for outputs.
        base_name = 'sheet_music'
        image_path = s_to_render.write('lily.png', fp=base_name)
        
        # Check if multiple pages were generated
        # music21 returns the path to the first file usually
        
        # Display logic
        generated_files = sorted(glob.glob(f"{base_name}*.png"))
        
        if not generated_files:
             # Fallback if the return path is specific and glob didn't catch it (unlikely with lily.png)
             if os.path.exists(str(image_path)):
                 generated_files = [str(image_path)]
        
        if generated_files:
            for img_file in generated_files:
                print(f"Displaying {img_file}...")
                display(Image(filename=img_file))
        else:
            print("No image files were generated.")

    except Exception as e:
        print(f"Error rendering sheet music: {e}")

# Replace with your actual output filename
output_midi = "outputs/inference/my_cover.mid"
if os.path.exists(output_midi):
    # Default to full score, but you can pass start_measure=1, end_measure=10 to limit it
    show_sheet_music(output_midi)
else:
    print(f"File not found: {output_midi}")

### Data Preparation (Optional)
If you are training the model, use `prepare.py`.

In [ ]:
!python prepare.py --start-from download